In [ ]:
import pandas as pd
import requests
import json

In [1]:
API_KEY = '7hlj8jHWV-jS57G2WGJEO0ROpEZAedHNxxd_4JMu1rdOW8ltXVO6Weeo-f_Tmpsm2_p-eG-Bq0wXMHHkerq33ul52MwZGynRQHwphIPSbeZl9BwZ_R-xuGg-Uqm-XHYx'

In [2]:
client_id = 'ZPsDRbHzglthukFTIg4SEA'

In [3]:
headers = {'Authorization': 'Bearer %s' % API_KEY}
url='https://api.yelp.com/v3/businesses/search'

In [4]:
zip_codes_SF = [94102, 94104, 94103, 94105, 94108, 94107, 94110, 
                94109, 94112, 94111, 94115, 94114, 94117, 94116,
                94118, 94121, 94123, 94122, 94124, 94127, 94126,
                94129, 94131, 94130, 94133, 94132, 94134, 94139,
                94143, 94151, 94159, 94158, 94188, 94177]

In [5]:
def get_values(key, list_dicts):
    values = []
    for dict_ in list_dicts:
        try:
            values.append(dict_[key])
        except:
            values.append(None)         
    return values

In [6]:
ids, names, rating, review_count, price, location, coordinates, transactions, categories, is_closed = [],[],[],[],[],[],[],[],[],[]
for zip_code in zip_codes_SF:
    params = {'term':'restaurants', 'location': str(zip_code), 'limit':'50'}
    req=requests.get(url, params=params, headers=headers)
    
    # proceed only if the status code is 200
    print('The status code is {}'.format(req.status_code))
    
    parsed = json.loads(req.text)
    businesses = parsed['businesses']


    ids.append(get_values('id', businesses))
    names.append(get_values('name', businesses))
    rating.append(get_values('rating', businesses))
    review_count.append(get_values('review_count', businesses))
    price.append(get_values('price', businesses))
    location.append(get_values('location', businesses))
    coordinates.append(get_values('coordinates', businesses))
    transactions.append(get_values('transactions', businesses))
    categories.append(get_values('categories', businesses))
    is_closed.append(get_values('is_closed', businesses))

The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200
The status code is 200


In [7]:
list_attrs = [ids, names, rating, review_count,
             price, transactions, is_closed]
name_attrs = ['ids', 'names', 'rating', 'review_count',
             'price', 'transactions', 'is_closed']

In [8]:
dict_flat = {}
for i, attrs in enumerate(list_attrs):
    dict_flat[name_attrs[i]] = [x for attr in attrs for x in attr]

In [9]:
alias_list = []
title_list = []

for category in categories:

    for cat in category:
        alias = []
        title = []
        for c in cat:
            try:
                alias.append(c["alias"])
                title.append(c["title"])
            except:
                pass
        alias_attr = ', '.join(alias)
        title_attr = ', '.join(title)

        alias_list.append(alias_attr) 
        title_list.append(title_attr) 

In [10]:
dict_flat['type_food'] = title_list
dict_flat['type_food_alias'] = alias_list

In [11]:
dict_flat['latitude'] = [x['latitude'] for coord in coordinates for x in coord]
dict_flat['longitude'] = [x['longitude'] for coord in coordinates for x in coord]

In [12]:
dict_flat['country'] = [x['country'] for loc in location for x in loc]
dict_flat['state'] = [x['state'] for loc in location for x in loc]
dict_flat['city'] = [x['city'] for loc in location for x in loc]
dict_flat['zip_code'] = [x['zip_code'] for loc in location for x in loc]

In [13]:
dict_flat.keys()

dict_keys(['ids', 'names', 'rating', 'review_count', 'price', 'transactions', 'is_closed', 'type_food', 'type_food_alias', 'latitude', 'longitude', 'country', 'state', 'city', 'zip_code'])

In [14]:
df = pd.DataFrame.from_dict(data=dict_flat)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1656 entries, 0 to 1655
Data columns (total 15 columns):
ids                1656 non-null object
names              1656 non-null object
rating             1656 non-null float64
review_count       1656 non-null int64
price              1233 non-null object
transactions       1656 non-null object
is_closed          1656 non-null bool
type_food          1656 non-null object
type_food_alias    1656 non-null object
latitude           1656 non-null float64
longitude          1656 non-null float64
country            1656 non-null object
state              1656 non-null object
city               1656 non-null object
zip_code           1656 non-null object
dtypes: bool(1), float64(3), int64(1), object(10)
memory usage: 182.8+ KB


In [17]:
def replace_empty_lists(list_entities):
    if len(list_entities) < 1:
        return None
    return list_entities

In [18]:
df['transactions_'] = df.transactions.apply(replace_empty_lists)

In [19]:
df.to_csv('../../data/interim/restaurants/yelp_sanfrancisco.csv')